# Real case - Acquisitor times (2 Peers)

### Imports

In [1]:
import threading
import requests
import json
import random
from acquisitor import *

### Global variables

In [2]:
API_ENDPOINT = "http://104.155.2.231:3000/api/"
NS = "ertis.uma.nuclear"

### Function

In [3]:
def addAcquisitionTest(num_acquisitors, num_tubes):
    acquisitor_threads = []
    acquisitions_per_worker = int(num_tubes/num_acquisitors)

    # Check for next acquisition id
    resource_url = f"{API_ENDPOINT}{NS}.Acquisition"
    r = requests.get(resource_url)
    next_id = len(r.json()) + 1

    # Create threads
    for i in range(num_acquisitors):
        thread = Acquisitor(i, f"Acquisitor-{i}", acquisitions_per_worker, next_id+acquisitions_per_worker*i, API_ENDPOINT, NS)
        acquisitor_threads.append(thread)

    # Start threads
    for i in range(num_acquisitors):
        acquisitor_threads[i].start()

    # Join threads
    for i in range(num_acquisitors):
        acquisitor_threads[i].join()
        
    # Print results
    min = acquisitor_threads[0].min
    avg = acquisitor_threads[0].avg
    max = acquisitor_threads[0].max
    acquisitor_threads[0].printResults()
    for i in range(1, num_acquisitors):
        if acquisitor_threads[i].min < min:
            min = acquisitor_threads[i].min

        avg = avg + acquisitor_threads[i].avg

        if acquisitor_threads[i].max > max:
            max = acquisitor_threads[i].max

        acquisitor_threads[i].printResults()

    print(f"MIN ADD ACQ --> {min}")
    print(f"AVG ADD ACQ --> {avg/num_acquisitors}")
    print(f"MAX ADD ACQ --> {max}")

### Execution of the test with one single acquisitor

In [4]:
addAcquisitionTest(1, 100) #Acquisitors, Acquisitions to do

Acquisitor Acquisitor-0 has started
Acquisitor Acquisitor-0 has finished
Acquisitor-0 - Fastest acquisition added in 2.3491342067718506 seconds
Acquisitor-0 - Slowest acquisition added in 2.5791475772857666 seconds
Acquisitor-0 - Average time adding acquisitions: 2.422208547592163 seconds
MIN ADD ACQ --> 2.3491342067718506
AVG ADD ACQ --> 2.422208547592163
MAX ADD ACQ --> 2.5791475772857666
